In [1]:
!pip install pymongo

In [14]:
#nomor 1 yang tugas
from pymongo import MongoClient, InsertOne, UpdateOne, DeleteOne

# Koneksi ke server MongoDB lokal
client = MongoClient('mongodb://localhost:27017/')
db = client['university_db']
courses_collection = db['courses']
courses_collection.delete_many({})
# Bulk insert of courses with student enrollments
operations = [
    InsertOne({'course': 'Math 101', 'enrollments': 30, 'department': 'Mathematics'}),
    InsertOne({'course': 'CS 102', 'enrollments': 25, 'department': 'Computer Science'}),
    InsertOne({'course': 'History 201', 'enrollments': 20, 'department': 'History'}),
    InsertOne({'course': 'Physics 202', 'enrollments': 15, 'department': 'Physics'})
]
courses_collection.bulk_write(operations)
print('Courses inserted successfully.')

Courses inserted successfully.


In [15]:
#nomor 1 yang latihan
# memperbaruhi data
update_operations = [
    UpdateOne({'course': 'Math 101'}, {'$set': {'enrollments': 35}})
]
#menghapus data 
delete_operations = [
    DeleteOne({'course': 'Physics 202'})
]
courses_collection.bulk_write(update_operations + delete_operations)
print("Update dan delete berhasil dijalankan.")

Update dan delete berhasil dijalankan.


In [16]:
print("Menampilkan Data Saat Ini:")
for course in courses_collection.find():
    print(course)

Menampilkan Data Saat Ini:
{'_id': ObjectId('690bf8462ea8560dd914b824'), 'course': 'Math 101', 'enrollments': 35, 'department': 'Mathematics'}
{'_id': ObjectId('690bf8462ea8560dd914b825'), 'course': 'CS 102', 'enrollments': 25, 'department': 'Computer Science'}
{'_id': ObjectId('690bf8462ea8560dd914b826'), 'course': 'History 201', 'enrollments': 20, 'department': 'History'}


In [18]:
# nomor 2 yang latihan
print("Kursus dengan jumlah mahasiswa > 20:")
for course in courses_collection.find({'enrollments': {'$gt': 20}}):
    print(course)

Kursus dengan jumlah mahasiswa > 20:
{'_id': ObjectId('690bf8462ea8560dd914b824'), 'course': 'Math 101', 'enrollments': 35, 'department': 'Mathematics'}
{'_id': ObjectId('690bf8462ea8560dd914b825'), 'course': 'CS 102', 'enrollments': 25, 'department': 'Computer Science'}


In [19]:
#nomor 2 yang tugas
# Query courses in Computer Science or Mathematics departments
print("Kursus dari departemen Computer Science atau Mathematics:")
for course in courses_collection.find({'department': {'$in': ['Computer Science', 'Mathematics']}}):
    print(course)

Kursus dari departemen Computer Science atau Mathematics:
{'_id': ObjectId('690bf8462ea8560dd914b824'), 'course': 'Math 101', 'enrollments': 35, 'department': 'Mathematics'}
{'_id': ObjectId('690bf8462ea8560dd914b825'), 'course': 'CS 102', 'enrollments': 25, 'department': 'Computer Science'}


In [21]:
#nomor 3 yang latihan 
# Average enrollment per department using aggregation
pipeline = [
    {'$group': {'_id': '$department', 'average_enrollment': {'$avg': '$enrollments'}}}
]

for result in courses_collection.aggregate(pipeline):
    print(result)

{'_id': 'Mathematics', 'average_enrollment': 35.0}
{'_id': 'History', 'average_enrollment': 20.0}
{'_id': 'Computer Science', 'average_enrollment': 25.0}


In [22]:
#nomor 3 yang tugas
# Maximum enrollment per department
pipeline = [
    {'$group': {'_id': '$department', 'max_enrollment': {'$max': '$enrollments'}}}
]

for result in courses_collection.aggregate(pipeline):
    print(result)

{'_id': 'Mathematics', 'max_enrollment': 35}
{'_id': 'Computer Science', 'max_enrollment': 25}
{'_id': 'History', 'max_enrollment': 20}


In [27]:
# nomor 4 yang latihan 
# Projection to rename fields
pipeline = [
    {'$project': {'course_name': '$course', 'department_name': '$department', 'enrollments': 1}}
]

for result in courses_collection.aggregate(pipeline):
    print(result)

{'_id': ObjectId('690bf8462ea8560dd914b824'), 'enrollments': 35, 'course_name': 'Math 101', 'department_name': 'Mathematics'}
{'_id': ObjectId('690bf8462ea8560dd914b825'), 'enrollments': 25, 'course_name': 'CS 102', 'department_name': 'Computer Science'}
{'_id': ObjectId('690bf8462ea8560dd914b826'), 'enrollments': 20, 'course_name': 'History 201', 'department_name': 'History'}


In [26]:
#nomor 4 yang tugas|
# Adding enrollment category field based on enrollments
pipeline = [
    {'$addFields': {'enrollment_category': {'$cond': {'if': {'$gt': ['$enrollments', 20]}, 'then': 'high', 'else': 'low'}}}}
]
for result in courses_collection.aggregate(pipeline):
    print(result)

{'_id': ObjectId('690bf8462ea8560dd914b824'), 'course': 'Math 101', 'enrollments': 35, 'department': 'Mathematics', 'enrollment_category': 'high'}
{'_id': ObjectId('690bf8462ea8560dd914b825'), 'course': 'CS 102', 'enrollments': 25, 'department': 'Computer Science', 'enrollment_category': 'high'}
{'_id': ObjectId('690bf8462ea8560dd914b826'), 'course': 'History 201', 'enrollments': 20, 'department': 'History', 'enrollment_category': 'low'}


In [28]:
#nomor 1. Homework
pipeline = [
    {'$group': {'_id': '$department', 'total_courses': {'$sum': 1}}}
]

for result in courses_collection.aggregate(pipeline):
    print(result)

{'_id': 'Mathematics', 'total_courses': 1}
{'_id': 'Computer Science', 'total_courses': 1}
{'_id': 'History', 'total_courses': 1}


In [34]:
#nomor 2 homework 
# Tambahkan data kursus
courses_collection.delete_many({})
courses_collection.insert_many([
    {'course': 'AI Fundamentals', 'enrollments': 32, 'department': 'Computer Science'},
    {'course': 'Web Development', 'enrollments': 27, 'department': 'Computer Science'},
    {'course': 'Database Systems', 'enrollments': 30, 'department': 'Computer Science'},
    {'course': 'Calculus II', 'enrollments': 33, 'department': 'Mathematics'}
])

# Pipeline agregasi untuk filter kursus dengan enrollments > 25 di "Computer Science"
pipeline = [
    {'$match': {'enrollments': {'$gt': 25}, 'department': 'Computer Science'}},  # Tahap penyaringan
    {'$group': {'_id': '$department', 'count': {'$sum': 1}}}                     # Tahap pengelompokan & perhitungan
]

# Eksekusi pipeline dan tampilkan hasil
for result in courses_collection.aggregate(pipeline):
    print(result)

{'_id': 'Computer Science', 'count': 3}


In [48]:
#nomor 3 homework 
courses_collection.insert_many([
    {'course': 'AI Fundamentals', 'enrollments': 32, 'department': 'Computer Science'},
    {'course': 'Web Development', 'enrollments': 27, 'department': 'Computer Science'},
    {'course': 'Database Systems', 'enrollments': 18, 'department': 'Computer Science'},
    {'course': 'Calculus II', 'enrollments': 33, 'department': 'Mathematics'}
])
students_collection = db['students']

students_collection.insert_many([
    {'name': 'Erlyn', 'enrolled_course': 'AI Fundamentals'},
    {'name': 'Abimanyu', 'enrolled_course': 'AI Fundamentals'},
    {'name': 'Priyo', 'enrolled_course': 'Web Development'},
    {'name': 'Ochi', 'enrolled_course': 'Database Systems'},
    {'name': 'Revi', 'enrolled_course': 'Calculus II'}
])
aggregation_result_3 = courses_collection.aggregate([
    {
        "$lookup": {
            "from": "students",              # Koleksi tujuan (students)
            "localField": "course",          # Field di koleksi courses
            "foreignField": "enrolled_course",# Field di koleksi students
            "as": "student_enrollments"      # Hasil join disimpan di sini
        }
    }
])

print("Hasil Lookup antara courses dan students:")
for result in aggregation_result_3:
    print(result)

Hasil Lookup antara courses dan students:
{'_id': ObjectId('690c04c62ea8560dd914b87f'), 'course': 'AI Fundamentals', 'enrollments': 32, 'department': 'Computer Science', 'student_enrollments': [{'_id': ObjectId('690c03f82ea8560dd914b844'), 'name': 'Alice', 'enrolled_course': 'AI Fundamentals'}, {'_id': ObjectId('690c03f82ea8560dd914b845'), 'name': 'Bob', 'enrolled_course': 'AI Fundamentals'}, {'_id': ObjectId('690c042d2ea8560dd914b84d'), 'name': 'Alice', 'enrolled_course': 'AI Fundamentals'}, {'_id': ObjectId('690c042d2ea8560dd914b84e'), 'name': 'Bob', 'enrolled_course': 'AI Fundamentals'}, {'_id': ObjectId('690c04552ea8560dd914b856'), 'name': 'Alice', 'enrolled_course': 'AI Fundamentals'}, {'_id': ObjectId('690c04552ea8560dd914b857'), 'name': 'Bob', 'enrolled_course': 'AI Fundamentals'}, {'_id': ObjectId('690c04642ea8560dd914b85f'), 'name': 'Alice', 'enrolled_course': 'AI Fundamentals'}, {'_id': ObjectId('690c04642ea8560dd914b860'), 'name': 'Bob', 'enrolled_course': 'AI Fundamentals'}